<a href="https://colab.research.google.com/github/KritskiE/ML/blob/main/aes-mars-disting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kada su n i p == 6 tada je najbolji LGBM i QDA (77% | 66%)


kada su n i p == 8 tada je najbolji AdaBoost   ( 77%)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree
from xgboost import XGBClassifier 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as pt
import seaborn as sb
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from os import listdir
from os.path import isfile, join
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense,Input,GlobalMaxPooling1D,GlobalMaxPooling2D
from tensorflow.keras.layers import Conv1D, MaxPooling1D,Embedding,Conv2D,MaxPooling2D
from tensorflow.keras.layers import LSTM,Dropout
from tensorflow.keras.models import Model
import tensorflow as tf
#print(tf.__version__)

In [ ]:
path='/content/drive/MyDrive/AES'
path2='/content/drive/MyDrive/MARS'

In [ ]:
def read_file(folder_path):
    with open(folder_path,"rb") as binary_file:
        data= binary_file.read()
    return data

In [ ]:
aes_samples = [f for f in listdir(path) if isfile(join(path, f))]
mars_samples= [f for f in listdir(path2) if isfile(join(path2, f))]

In [ ]:
print("AES:",len(aes_samples),"| MARS:",len(mars_samples))


AES: 30 | MARS: 30


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bitstring
from bitstring import BitArray

Pretvaranje sifrata u binarni niz

In [ ]:
X=[]
X_byteint=[]
X_bits=[]

X1=[]
X1_byteint=[]
X1_bits=[]

In [ ]:
def tobitarray(num):
    s=format(num,'b').zfill(8)
    return np.array([int(c) for c in s])

In [ ]:
#aes
for sample in aes_samples:
    folder_path= join(path,sample)
    cc=read_file(folder_path)
    X.append(cc)
    X_byteint.append(list(cc))
    X_bits.append([tobitarray(num) for num in list(cc)])
#mars    
for sample in mars_samples:
    folder_path= join(path2,sample)
    cc=read_file(folder_path)
    X1.append(cc)
    X1_byteint.append(list(cc))
    X1_bits.append([tobitarray(num) for num in list(cc)])

In [ ]:
X_bits_flatten = [np.array(b).ravel() for b in X_bits]
X1_bits_flatten = [np.array(b).ravel() for b in X1_bits]

Dimenzije binarnih podataka

In [ ]:
print(np.array(X_bits_flatten).shape)
print(np.array(X1_bits_flatten).shape)

(30, 65536)
(30, 65536)


In [ ]:
df1 = pd.DataFrame(X_bits_flatten)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,65496,65497,65498,65499,65500,65501,65502,65503,65504,65505,65506,65507,65508,65509,65510,65511,65512,65513,65514,65515,65516,65517,65518,65519,65520,65521,65522,65523,65524,65525,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535
0,1,0,1,0,0,0,0,1,1,0,1,0,0,1,1,0,1,1,0,0,1,1,1,0,1,1,0,0,0,1,1,0,1,1,0,1,0,1,1,0,...,0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,1
1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,1,1,1,1,1,1,0,1,0,...,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,1,1,0,0,...,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,0,1,0,1,0,1,0,0,0,0,0
3,1,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,1,...,0,1,0,1,1,0,0,1,0,0,1,1,0,0,1,0,1,0,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0,1,0,1,1,0,1,1
4,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,0,...,0,1,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0


In [ ]:
df2 = pd.DataFrame(X1_bits_flatten)
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,65496,65497,65498,65499,65500,65501,65502,65503,65504,65505,65506,65507,65508,65509,65510,65511,65512,65513,65514,65515,65516,65517,65518,65519,65520,65521,65522,65523,65524,65525,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535
0,0,1,1,1,0,1,0,0,0,1,1,0,1,1,1,1,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,0,0,1,0,0,...,1,1,1,1,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,1,0,1,1,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,1,1,1,0,1,0,0,1,1,1,0,0,0,1,...,0,1,1,0,0,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
2,1,0,0,1,0,1,0,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,1,1,0,1,1,0,0,1,1,1,1,0,1,0,0,0,1,...,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,1,1,1,0,1,0,1,1,1,1,0,1,1,1,0,1,1,0
3,0,1,0,1,1,0,1,1,1,1,0,1,1,0,0,0,1,1,1,0,1,1,0,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,...,1,0,0,0,0,1,0,0,1,1,1,0,1,0,0,1,0,0,1,1,1,1,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0
4,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,1,1,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1


Ngrami - smanjivanje dimenzionalnosti

In [ ]:
def seq_ngrams(xs, n, p=1):
    return [xs[i:i+n] for i in range(0,len(xs)-n+1,p)]

def ngrams_dec(xs, n, p=1):
  data=[BitArray(ngram).uint for i in range(xs.shape[0]) for ngram in seq_ngrams(xs[i,:],n,p)]
  return pd.DataFrame(np.reshape(np.array(data),(xs.shape[0],-1)))


def ngrams_dict(xs, n, p=1):
  data=[BitArray(ngram).uint for i in range(xs.shape[0]) for ngram in seq_ngrams(xs[i,:],n,p)]
  unique_data = pd.unique(data) #np.unique vraca sortiran data niz dok pd.unique vraca jedinstvene podatke po redosledu pojavljivanja u data nizu
  dataend=[np.where(d==unique_data)[0] for d in data]
  return pd.DataFrame(np.reshape(np.array(dataend),(xs.shape[0],-1)))

In [ ]:
x1=df1.to_numpy()
x2=df2.to_numpy()

In [ ]:
# korak ngrama
n=8;p=8
df1_dict = ngrams_dict(x1,n,p)
df2_dict = ngrams_dict(x2,n,p)

In [ ]:
AES = df1_dict.to_numpy()
MARS = df2_dict.to_numpy()

In [ ]:
print(AES.shape, MARS.shape)
#dimenzije dataframe-ova nakon upotrebe ngrama

(30, 8192) (30, 8192)


Dodavanje label-a za razlikovanje sifrata

In [ ]:
t=np.concatenate((np.full(30,1),np.full(30,5)),axis=None)

Spajanje dataframe-ova

In [ ]:
df = pd.concat([df1_dict, df2_dict])

In [ ]:
#data_dmatrix = xgb.DMatrix(data=X,label=t)

In [ ]:
X=df.to_numpy()

In [ ]:
X.shape

(60, 8192)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, t, test_size=0.3, random_state=1, shuffle=True)
X_train.shape, X_test.shape, y_train.shape,y_test.shape

((42, 8192), (18, 8192), (42,), (18,))

In [ ]:
#pca = PCA(random_state = 2019)
#X_pca = pca.fit_transform(X)

In [ ]:
#pt.figure(figsize=(10,7))
#pt.plot(np.cumsum(pca.explained_variance_ratio_))
#pt.xlabel('number of components')
#pt.ylabel('cumulative explained variance');

### RandomForest

In [ ]:
from sklearn import metrics
clf=RandomForestClassifier(n_estimators=900)
clf.fit(X_train,y_train)
y_predict=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test,y_predict))

Accuracy: 0.3333333333333333


### SVC

In [ ]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.4444444444444444


### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_predict = logreg.predict(X_test)

In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_predict)
cnf_matrix

array([[ 7,  0],
       [10,  1]])

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_predict))
print("Precision:",metrics.precision_score(y_test, y_predict))

Accuracy: 0.4444444444444444
Precision: 0.4117647058823529


### AdaBoostClassifier 
(najbolji rezultat)

In [ ]:

abc = AdaBoostClassifier(n_estimators=100,
                         learning_rate=1)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

NameError: ignored

### LGBM 

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred=clf.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

### QuadraticDiscriminantAnalysis

In [ ]:
 from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train,y_train)
y_pred=qda.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
print("Accuracy: ", metrics.accuracy_score(y_pred,y_test))

### DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

### ExtraTreeClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 100,
                                        criterion ='entropy', max_features = 24)
  
# Training the model
extra_tree_forest.fit(X_train, y_train)
y_pred=extra_tree_forest.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

1.QDC
2.SGD
3.RandomForest
4.DecisionTree

### LazyPredict

In [ ]:
%%time
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None,
                     random_state=12,
                     classifiers='all',
                    )
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

In [ ]:
model_dictionary = clf.provide_models(X_train,X_test,y_train,y_test)
models

#### SGD classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
clf = SGDClassifier(loss="log", penalty="l2")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalMaxPooling2D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
import tensorflow as tf
print(tf.__version__)

In [ ]:
# there is also a pandas_input_fn we'll see in the exercise!!
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10], n_classes=3,feature_columns=feat_cols)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalMaxPooling2D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
import tensorflow as tf
print(tf.__version__)

2.7.0


In [ ]:
T =X_train.shape[1]

In [ ]:
# We get to choose embedding dimensionality
D = 15
#@V = 8
# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

i = Input(shape=(T,))
e = Embedding(V + 1, D)(i)
x = Conv1D(32, 3, activation='relu')(e)
x = MaxPooling1D(3)(x)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8192)]            0         
                                                                 
 embedding (Embedding)       (None, 8192, 15)          135       
                                                                 
 conv1d (Conv1D)             (None, 8190, 32)          1472      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2730, 32)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 2728, 64)          6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 909, 64)          0         
 1D)                                                         

In [ ]:
# Compile and fit
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


print('Training model...')
r = model.fit(
  X_train,
  y_train,
  epochs=15,
  validation_data=(X_test, y_test)
)

Training model...
Epoch 1/15


InvalidArgumentError: ignored